# Convnet Model and Training

This aim of notebook is to define the siamese model that will be used for the Humpback Whale challenge. It will also run the model to trainin it agains our previously generated training and validation dataset and also create the results agains a test dataset.

## 0. Module Imports 

In [1]:
import numpy as np
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import matplotlib.pyplot as plt

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 1. Load Training, Validation and Test datasets

In [2]:
pickle_file = '../data/Siamese_dataset.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_X1_dataset = save['train_X1_dataset']
    train_X2_dataset = save['train_X2_dataset']
    train_labels = save['train_labels']
    valid_X1_dataset = save['valid_X1_dataset']
    valid_X2_dataset = save['valid_X2_dataset']
    valid_labels = save['valid_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_X1_dataset.shape, train_X2_dataset.shape, train_labels.shape)
    print('Validation set', valid_X1_dataset.shape, valid_X2_dataset.shape, valid_labels.shape)

Training set (15607, 44, 82) (15607, 44, 82) (15607,)
Validation set (2755, 44, 82) (2755, 44, 82) (2755,)


##  2. Declaration of Global Constants
Now we declare all the global constants in hee in orde to have them centralized.
This way we can easyly tune our model.

In [3]:
## Image parameters
NUM_CHANNELS = 1
IMAGE_HEIGHT = 44
IMAGE_WIDHT = 82

#Convnet parameters
BETA = 0.005
PATCH_1 = 7
PATCH_2 = 6
DEPTH_1 = 15
DEPTH_2 = 45
NUM_HIDEN = 1024
NUM_LABELS = 1
MARGIN = 8.0


#Training parameters
BATCH_SIZE = 128
NUM_STEPS = 1501

## 3. Reformat datasets into a TensorFlow-friendly shape:

- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.



In [4]:
num_labels = 1

def reformat(dataset1, dataset2, labels):
  dataset1 = dataset1.reshape(
    (-1, IMAGE_HEIGHT, IMAGE_WIDHT, NUM_CHANNELS)).astype(np.float32)
  dataset2 = dataset2.reshape(
    (-1, IMAGE_HEIGHT, IMAGE_WIDHT, NUM_CHANNELS)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset1, dataset2, labels
train_X1_dataset, train_X2_dataset, train_labels = reformat(train_X1_dataset, train_X2_dataset, train_labels)
valid_X1_dataset, valid_X2_dataset, valid_labels = reformat(valid_X1_dataset, valid_X2_dataset, valid_labels)
print('Training set', train_X1_dataset.shape, train_X2_dataset.shape, train_labels.shape)
print('Validation set', valid_X1_dataset.shape, valid_X2_dataset.shape, valid_labels.shape)

Training set (15607, 44, 82, 1) (15607, 44, 82, 1) (15607, 1)
Validation set (2755, 44, 82, 1) (2755, 44, 82, 1) (2755, 1)


## 4. Definition of auxiliar functions

### 4.1. Accuracy computation

In [5]:
def accuracy(predictions, labels):
    aux = predictions < MARGIN
    equal = np.equal(aux[:] , labels[:, 0])
    great = np.greater(aux[:] , labels[:, 0])
    less = np.less(aux[:] , labels[:, 0])
    acc = (100.0 * np.sum(equal.astype(np.float32))
          / predictions.shape[0])
    false_pos = np.sum(great.astype(np.float32))
    false_neg = np.sum(less.astype(np.float32))
    return acc, false_pos, false_neg

def print_means(distance, labels):
    print(distance.shape)
    print(labels.shape)
    positives = np.where(labels[:, 0] > 0)
    negatives = np.where(labels[:, 0] == 0)
    print(positives[0].shape)
    print(negatives[0].shape)
    mean_p_distance = np.mean(distance[positives[0]])
    mean_n_distance = np.mean(distance[negatives[0]])
    print('Mean negative distance ' +  str(mean_n_distance) + ' mean positive distance ' + str(mean_p_distance))

## 5. Network declaration
We will use tensorflow

In [6]:

siamese_model = tf.Graph()

with siamese_model.as_default():

  # Input data.
    tf_train_X1_dataset = tf.placeholder(\
          tf.float32, shape=(BATCH_SIZE, IMAGE_HEIGHT, IMAGE_WIDHT, NUM_CHANNELS))
    tf_train_X2_dataset = tf.placeholder(\
          tf.float32, shape=(BATCH_SIZE, IMAGE_HEIGHT, IMAGE_WIDHT, NUM_CHANNELS))
    tf_train_labels = tf.placeholder(tf.float32, shape=(BATCH_SIZE, NUM_LABELS))
    tf_valid_X1_dataset = tf.constant(valid_X1_dataset)
    tf_valid_X2_dataset = tf.constant(valid_X2_dataset)
    #tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = {
      'layer1': tf.Variable(tf.truncated_normal(\
              [PATCH_1, PATCH_1, NUM_CHANNELS, DEPTH_1], stddev=0.1)),
      'layer2': tf.Variable(tf.truncated_normal(\
              [PATCH_2, PATCH_2, DEPTH_1, DEPTH_2], stddev=0.1)),
      'layer3': tf.Variable(tf.truncated_normal(\
              [IMAGE_HEIGHT // 4 * (IMAGE_WIDHT // 4 + 1) * DEPTH_2, NUM_HIDEN], stddev=0.1)),
      'layer4': tf.Variable(tf.truncated_normal(\
              [NUM_HIDEN, 2], stddev=0.1))
      }
    biases = {
        'layer1' : tf.Variable(tf.zeros([DEPTH_1])),
        'layer2' : tf.Variable(tf.constant(1.0, shape=[DEPTH_2])),
        'layer3' : tf.Variable(tf.constant(1.0, shape=[NUM_HIDEN])),
        'layer4' : tf.Variable(tf.constant(1.0, shape=[1]))
        }
  
  # Model.
    def model(data):
        conv = tf.nn.conv2d(data, weights['layer1'], [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + biases['layer1'])
        conv = tf.nn.conv2d(hidden, weights['layer2'], [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + biases['layer2'])
        shape = hidden.get_shape().as_list()
        print(shape)
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, weights['layer3']) + biases['layer3'])
        return tf.matmul(hidden, weights['layer4']) + biases['layer4']
    
    
  # Training computation.
    with tf.variable_scope("siamese") as scope:
        logits1 = model(tf_train_X1_dataset)
        scope.reuse_variables()
        logits2 = model(tf_train_X2_dataset)
        
    with tf.variable_scope("siamese_val") as scope:
        val_logits1 = model(tf_valid_X1_dataset)
        scope.reuse_variables()
        val_logits2 = model(tf_valid_X2_dataset)
        
    def loss(y):
        labels_t = y
        labels_f = tf.subtract(1.0, y, name="1-yi")          # labels_ = !labels;
        eucd2 = tf.pow(tf.subtract(logits1, logits2), 2)
        eucd2 = tf.reduce_sum(eucd2, 1)
        eucd = tf.sqrt(eucd2+1e-6, name="eucd")
        C = tf.constant(MARGIN, name="C")
        # yi*||CNN(p1i)-CNN(p2i)||^2 + (1-yi)*max(0, C-||CNN(p1i)-CNN(p2i)||^2)
        pos = tf.multiply(labels_t, eucd2, name="yi_x_eucd2")
        # neg = tf.multiply(labels_f, tf.subtract(0.0,eucd2), name="yi_x_eucd2")
        # neg = tf.multiply(labels_f, tf.maximum(0.0, tf.subtract(C,eucd2)), name="Nyi_x_C-eucd_xx_2")
        neg = tf.multiply(labels_f, tf.pow(tf.maximum(tf.subtract(C, eucd), 0), 2), name="Nyi_x_C-eucd_xx_2")
        losses = tf.add(pos, neg, name="losses")
        loss = tf.reduce_mean(losses, name="loss")
        return loss
        
    loss = loss(tf_train_labels)
    
  # Optimizer.
    optimizer = tf.train.AdamOptimizer().minimize(loss)
  
  
  # Predictions for the training, validation, and test data.
    train_prediction = tf.reduce_sum(tf.pow(tf.subtract(logits1, logits2), 2),1)
    print(train_prediction.shape)
    valid_prediction = tf.reduce_sum(tf.pow(tf.subtract(val_logits1, val_logits2), 2),1)
    print(valid_prediction.shape)
    #test_prediction = tf.nn.softmax(model(tf_test_dataset))


[128, 11, 21, 45]
[128, 11, 21, 45]
[2755, 11, 21, 45]
[2755, 11, 21, 45]
(128,)
(2755,)


## 6. Model training

In [7]:
#Plots
P_FREQ = 50
losses = []
accuracies = []
rates = []

with tf.Session(graph=siamese_model) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(NUM_STEPS):
    offset = (step * BATCH_SIZE) % (train_labels.shape[0] - BATCH_SIZE)
    batch_data_X1 = train_X1_dataset[offset:(offset + BATCH_SIZE), :, :, :]
    batch_data_X2 = train_X2_dataset[offset:(offset + BATCH_SIZE), :, :, :]
    batch_labels = train_labels[offset:(offset + BATCH_SIZE), :]
    feed_dict = {tf_train_X1_dataset : batch_data_X1, tf_train_X2_dataset : batch_data_X2, \
                 tf_train_labels : batch_labels}
    _, _loss, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if np.isnan(_loss):
        print('Model diverged with loss = NaN')
        quit()
        
    #Plot variables
    if (step % P_FREQ == 0):
      losses.append(_loss)
      distances = valid_prediction.eval()
      acc, fp, fn = accuracy(distances, valid_labels)
      accuracies.append(acc)
      rates.append(float(fp/fn))
    
    if (step % 50 == 0):
      print('Minibatch loss at step {} {}'.format(step, _loss))
      a = accuracy(predictions, batch_labels)
      print('Minibatch accuracy: {} false positives: {} false negatives: {}'.format(a[0], a[1], a[2] ) )
      a = accuracy(valid_prediction.eval(), valid_labels)
      print('Validation accuracy: {} false positives: {} false negatives: {}'.format(a[0], a[1], a[2] ) )
      print_means(valid_prediction.eval(), valid_labels)
#print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

# Show the results.
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
plt.subplots_adjust(wspace=.8)
fig.set_size_inches(10, 4)
ax1.plot(range(0, NUM_STEPS, P_FREQ), losses)
ax1.set_ylabel("Loss")
ax1.set_xlabel("Training steps")
ax2.plot(range(0, NUM_STEPS, P_FREQ), accuracies)
ax2.set_ylabel("Validation Accuracy [%]")
ax2.set_xlabel("Training steps")
ax3.plot(range(0, NUM_STEPS, P_FREQ), rates)
ax3.set_ylabel("FP / FN [%]")
ax3.set_xlabel("Training steps")
plt.show()


Initialized
Minibatch loss at step 0 28.7582817078
Minibatch accuracy: 31.25 false positives: 36.0 false negatives: 52.0
Validation accuracy: 42.5045372051 false positives: 197.0 false negatives: 1387.0
(2755,)
(2755, 1)
(1406,)
(1349,)
Mean negative distance 482.16696 mean positive distance 1702.8926
Minibatch loss at step 50 19.1723957062
Minibatch accuracy: 40.625 false positives: 41.0 false negatives: 35.0
Validation accuracy: 45.55353902 false positives: 795.0 false negatives: 705.0
(2755,)
(2755, 1)
(1406,)
(1349,)
Mean negative distance 10.6664715 mean positive distance 12.554004
Minibatch loss at step 100 20.4884872437
Minibatch accuracy: 44.53125 false positives: 39.0 false negatives: 32.0
Validation accuracy: 41.8511796733 false positives: 784.0 false negatives: 818.0
(2755,)
(2755, 1)
(1406,)
(1349,)
Mean negative distance 10.817733 mean positive distance 14.938953
Minibatch loss at step 150 18.6811771393
Minibatch accuracy: 42.96875 false positives: 34.0 false negatives: 39

KeyboardInterrupt: 

In [ ]:
print(valid_labels.shape)

In [ ]:
print(predictions.shape)

In [ ]:
print(hey)

In [ ]:
a = hey < 5.0
print(a.astype(np.float32))

In [ ]:
print(_valid_labels)
print(valid_labels)

In [ ]:
print(a[23].astype(np.float32))
print(valid_labels[23])

In [ ]:
def accuracy_dbg(predictions, labels, print_nr = 0):
    aux = predictions < MARGIN
    equal = np.equal(aux[:] , labels[:,0])
    print(predictions[print_nr])
    print(labels[print_nr])
    print(aux[print_nr].astype(np.float32))
    print(aux.shape)
    print(labels.shape)
    print(equal.shape)
    print(predictions.shape)
    acc = (100.0 * np.sum(equal.astype(np.float32))
          / predictions.shape[0])
    false_pos = 12
    false_neg = 122
    return acc, false_pos, false_neg
a = accuracy_dbg(hey, valid_labels, 24)
print(a)
print(" {} {} {}".format(a[0], a[1], a[2]))

In [ ]:
a = np.array([0, 1, 1, 1, 1, 0, 0, 0])
b = np.where(a == 0)
c = a[b]
print(c)